In [1]:
# -*- coding: utf-8 -*-
"""
Dash: Choropleth nur Thüringen + Sachsen-Anhalt (Landkreise)

Start:
    python bruecken_th_sa_app.py
Browser:
    http://127.0.0.1:8052
"""

import json
import re
import unicodedata
from pathlib import Path
from typing import List

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import plotly.graph_objects as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output


# ------------------------------------------------------------
# Konfiguration
# ------------------------------------------------------------
csv_path = Path("data/original_bridge_statistic_germany.csv")
geo_path = Path("landkreise_simplify200.geojson")
states_path = Path("bundeslaender_simplify0.geojson")
AGE_REF_YEAR = 2025

FOCUS_STATES = ["Thüringen", "Sachsen-Anhalt"]  # muss exakt zu deinem GeoJSON passen


# ------------------------------------------------------------
# Helpers
# ------------------------------------------------------------
def pick_first(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    return None


def norm_name(s: str) -> str:
    s = str(s).strip().lower()
    s = s.replace("ä", "ae").replace("ö", "oe").replace("ü", "ue").replace("ß", "ss")
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    s = re.sub(r"\b(landkreis|kreisfreie stadt|kreis|stadt|region|lkr\.?)\b", "", s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()


def make_points_from_lonlat(dfin, lon_col="x2", lat_col="y2"):
    lon = pd.to_numeric(dfin[lon_col].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    lat = pd.to_numeric(dfin[lat_col].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    ok = lon.notna() & lat.notna()
    g = gpd.GeoDataFrame(
        dfin[ok].copy(),
        geometry=[Point(xy) for xy in zip(lon[ok], lat[ok])],
        crs=4326,
    )
    g["lon"] = g.geometry.x
    g["lat"] = g.geometry.y
    return g


def make_points_from_utm(dfin, x_col="X", y_col="Y", epsg=25832):
    xx = pd.to_numeric(dfin[x_col].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    yy = pd.to_numeric(dfin[y_col].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    ok = xx.notna() & yy.notna()
    g = gpd.GeoDataFrame(
        dfin[ok].copy(),
        geometry=[Point(xy) for xy in zip(xx[ok], yy[ok])],
        crs=epsg,
    ).to_crs(4326)
    g["lon"] = g.geometry.x
    g["lat"] = g.geometry.y
    return g


# ------------------------------------------------------------
# 1) Geodaten laden
# ------------------------------------------------------------
gdf_kreise = gpd.read_file(geo_path)
gdf_states = gpd.read_file(states_path)

# CRS auf WGS84
if gdf_kreise.crs is None:
    gdf_kreise = gdf_kreise.set_crs(4326)
else:
    gdf_kreise = gdf_kreise.to_crs(4326)

if gdf_states.crs is None:
    gdf_states = gdf_states.set_crs(4326)
else:
    gdf_states = gdf_states.to_crs(4326)

# Spalten identifizieren
colK_name = pick_first(gdf_kreise.columns, ["GEN", "NAME_3", "NAME", "GEN_NAME", "KREIS"])
colK_ags = pick_first(gdf_kreise.columns, ["AGS", "AGS_0", "RS", "RS_0", "ID_3"])

if colK_name is None:
    raise ValueError(f"Konnte keinen Kreisnamen finden. Spalten: {list(gdf_kreise.columns)}")

colS_name = pick_first(gdf_states.columns, ["GEN", "NAME", "STATE_NAME"])
colS_id = pick_first(gdf_states.columns, ["RS", "RS_0", "AGS", "AGS_0", "ID"])
if colS_name is None:
    raise ValueError(f"Konnte keinen Bundesländer-Namen finden. Spalten: {list(gdf_states.columns)}")
if colS_id is None:
    colS_id = colS_name

gdf_states = gdf_states.copy()
gdf_states["state_id"] = gdf_states[colS_id].astype(str)

# Fokus-Bundesländer + Fokus-Landkreise (räumlich)
gdf_states_focus = gdf_states[gdf_states[colS_name].isin(FOCUS_STATES)].copy()
if len(gdf_states_focus) == 0:
    raise ValueError(
        f"FOCUS_STATES matcht nicht. Verfügbare Werte in '{colS_name}': "
        f"{sorted(gdf_states[colS_name].astype(str).unique())}"
    )

focus_union = gdf_states_focus.unary_union

# Nur Landkreise, deren "Innenpunkt" innerhalb des Fokus liegt
rp = gdf_kreise.geometry.representative_point()
gdf_kreise_focus = gdf_kreise[rp.within(focus_union)].copy()

# ------------------------------------------------------------
# 2) Brückendaten laden + Punkte + Join
# ------------------------------------------------------------
df = pd.read_csv(csv_path, sep=";", decimal=",", dtype=str, keep_default_na=False)

for sp in [
    "Zustandsnote", "Substanzkennzahl",
    "x2", "y2", "X", "Y",
    "Kreis", "Baujahr Überbau", "Traglastindex",
    "Zugeordneter Sachverhalt vereinfacht", "Baustoffklasse", "Bauwerksname"
]:
    if sp not in df.columns:
        df[sp] = ""

df["Zustandsnote"] = pd.to_numeric(df["Zustandsnote"].astype(str).str.replace(",", ".", regex=False), errors="coerce")
df["Substanzkennzahl"] = pd.to_numeric(df["Substanzkennzahl"].astype(str).str.replace(",", ".", regex=False), errors="coerce")
df["Baujahr Überbau"] = pd.to_numeric(df["Baujahr Überbau"].astype(str).str.replace(",", ".", regex=False), errors="coerce")

# Punkte bevorzugt x2/y2, sonst X/Y
gdf_pts = make_points_from_lonlat(df, "x2", "y2")
used_coords = "WGS84 (x2/y2)"
if len(gdf_pts) == 0:
    gdf_pts = make_points_from_utm(df, "X", "Y", epsg=25832)
    used_coords = "UTM32 (EPSG:25832)"

gdf_pts["Baujahr"] = pd.to_numeric(gdf_pts["Baujahr Überbau"], errors="coerce")

# räumlicher Join auf Fokus-Kreise
join_cols = [colK_name, "geometry"] if colK_ags is None else [colK_name, colK_ags, "geometry"]
joined = gpd.sjoin(
    gdf_pts,
    gdf_kreise_focus[join_cols],
    how="left",
    predicate="within",
)

# nur Punkte im Fokus behalten
joined = joined[joined[colK_name].notna()].copy()
gdf_pts_focus = gdf_pts.loc[joined.index].copy()

# Basis für Aggregation: räumlicher Join
base = joined.copy()
base["Alter_Überbau"] = AGE_REF_YEAR - base["Baujahr Überbau"]

roman_map = {"I": 1, "II": 2, "III": 3, "IV": 4, "V": 5}
base["Traglastindex_norm"] = base["Traglastindex"].astype(str).str.upper().str.strip().map(roman_map)

# Aggregation pro Landkreisname
agg = (
    base.dropna(subset=[colK_name])
    .groupby(colK_name, as_index=False)
    .agg(
        n_bridges=("Bauwerksname", "size"),
        mean_zust=("Zustandsnote", "mean"),
        mean_trag=("Traglastindex_norm", "mean"),
        mean_age=("Alter_Überbau", "mean"),
        mean_substanz=("Substanzkennzahl", "mean"),
    )
)

# In GeoDF mergen
gplot = gdf_kreise_focus.merge(agg, left_on=colK_name, right_on=colK_name, how="left")

# stabile IDs
if colK_ags and colK_ags in gplot.columns:
    gplot["id"] = gplot[colK_ags].astype(str)
else:
    gplot["id"] = gplot[colK_name].astype(str)

# GeoJSONs
geojson_kreise = json.loads(gplot[["id", "geometry"]].to_json())
geojson_states = json.loads(gdf_states_focus[["state_id", "geometry"]].to_json())


# ------------------------------------------------------------
# 3) Dash UI
# ------------------------------------------------------------
choropleth_mode_options = [
    {"label": "Zustandsnote",        "value": "zustand"},
    {"label": "Traglastindex",       "value": "trag"},
    {"label": "Durchschnittsalter",  "value": "alter"},
    {"label": "Substanzkennzahl",    "value": "substanz"},
]

layer_options = [
    {"label": "Bundesländer-Grenzen", "value": "states"},
    {"label": "Landkreis-Grenzen",    "value": "kreise"},
    {"label": "Brückenpunkte",        "value": "points"},
]

app = dash.Dash(__name__)

app.layout = html.Div(
    style={"display": "flex", "height": "100vh", "fontFamily": "sans-serif"},
    children=[
        html.Div(
            style={"width": "22%", "padding": "10px", "borderRight": "1px solid #ccc", "overflowY": "auto"},
            children=[
                html.H3("Fokus"),
                html.Div(", ".join(FOCUS_STATES)),
                html.Hr(),
                html.H3("Flächenanzeige"),
                dcc.RadioItems(
                    id="choropleth-mode",
                    options=choropleth_mode_options,
                    value="zustand",
                    labelStyle={"display": "block"},
                ),
                html.Hr(),
                html.H3("Ebenen"),
                dcc.Checklist(
                    id="layer-checklist",
                    options=layer_options,
                    value=["states", "kreise"],  # Punkte standardmäßig aus
                    labelStyle={"display": "block"},
                ),
                html.Hr(),
                html.Div(f"Koordinaten: {used_coords}", style={"fontSize": "12px", "color": "#555"}),
            ],
        ),
        html.Div(
            style={"width": "78%", "padding": "10px"},
            children=[dcc.Graph(id="map-figure", style={"height": "100%"})],
        ),
    ],
)


# ------------------------------------------------------------
# 4) Figure Builder
# ------------------------------------------------------------
def build_figure(choropleth_mode: str, layers: List[str]):
    fig = go.Figure()

    # Metrik wählen
    if choropleth_mode == "trag":
        metric_col = "mean_trag"
        metric_label = "Ø Traglastindex"
        fmt = ".2f"
    elif choropleth_mode == "alter":
        metric_col = "mean_age"
        metric_label = "Ø Alter (Jahre)"
        fmt = ".1f"
    elif choropleth_mode == "substanz":
        metric_col = "mean_substanz"
        metric_label = "Ø Substanzkennzahl"
        fmt = ".2f"
    else:
        metric_col = "mean_zust"
        metric_label = "Ø Zustandsnote"
        fmt = ".2f"

    # Choropleth (deckend)
    g_valid = gplot[gplot[metric_col].notna()].copy()
    g_miss = gplot[gplot[metric_col].isna()].copy()

    if len(g_miss) > 0:
        fig.add_trace(
            go.Choropleth(
                geojson=geojson_kreise,
                locations=g_miss["id"],
                z=[0] * len(g_miss),
                featureidkey="properties.id",
                name="keine Daten",
                showlegend=True,
                showscale=False,
                colorscale=[[0, "lightgrey"], [1, "lightgrey"]],
                hoverinfo="skip",
                marker_opacity=1.0,
                marker_line_color="rgba(0,0,0,0)",
                marker_line_width=0.0,
            )
        )

    if len(g_valid) > 0:
        fig.add_trace(
            go.Choropleth(
                geojson=geojson_kreise,
                locations=g_valid["id"],
                z=g_valid[metric_col],
                featureidkey="properties.id",
                name=metric_label,
                showlegend=True,
                showscale=True,
                colorbar_title=metric_label,
                colorscale="RdYlGn_r",
                marker_opacity=1.0,
                hovertemplate=f"<b>%{{text}}</b><br>{metric_label}: %{{z:{fmt}}}<extra></extra>",
                text=g_valid[colK_name],
                marker_line_color="rgba(0,0,0,0)",
                marker_line_width=0.0,
            )
        )

    # Bundesländer-Grenzen (weiß)
    if "states" in layers:
        fig.add_trace(
            go.Choropleth(
                geojson=geojson_states,
                locations=gdf_states_focus["state_id"],
                z=[0] * len(gdf_states_focus),
                featureidkey="properties.state_id",
                name="Bundesländer",
                showlegend=True,
                showscale=False,
                hoverinfo="skip",
                colorscale=[[0, "rgba(0,0,0,0)"], [1, "rgba(0,0,0,0)"]],
                marker_line_color="rgba(130,130,10,0.95)",
                marker_line_width=1.7,
            )
        )

    # Landkreis-Grenzen (grau)
    if "kreise" in layers:
        fig.add_trace(
            go.Choropleth(
                geojson=geojson_kreise,
                locations=gplot["id"],
                z=[0] * len(gplot),
                featureidkey="properties.id",
                name="Landkreis-Grenzen",
                showlegend=False,
                showscale=False,
                hoverinfo="skip",
                colorscale=[[0, "rgba(0,0,0,0)"], [1, "rgba(0,0,0,0)"]],
                marker_line_color="rgba(170,170,170,0.9)",
                marker_line_width=0.4,
            )
        )

    # Brückenpunkte (optional)
    if "points" in layers and len(gdf_pts_focus) > 0:
        fig.add_trace(
            go.Scattergeo(
                lon=gdf_pts_focus["lon"],
                lat=gdf_pts_focus["lat"],
                mode="markers",
                name="Brücken",
                marker=dict(size=1.4, color="rgba(20,20,20,1.0)", opacity=0.75),
                showlegend=True,
            )
        )

    # Zoom auf Fokus
    fig.update_geos(
        fitbounds="locations",
        visible=False,
        projection_type="mercator",
    )

    fig.update_layout(
        margin={"r": 20, "t": 60, "l": 20, "b": 20},
        legend=dict(x=0.02, y=0.98, bgcolor="rgba(255,255,255,0.85)"),
        title=f"Thüringen + Sachsen-Anhalt – {metric_label} pro Landkreis<br><sup>Koordinaten: {used_coords}</sup>",
    )

    return fig


@app.callback(
    Output("map-figure", "figure"),
    [Input("choropleth-mode", "value"), Input("layer-checklist", "value")],
)
def update_map(mode, layers):
    if mode is None:
        mode = "zustand"
    if layers is None:
        layers = []
    return build_figure(mode, layers)


if __name__ == "__main__":
    app.run(debug=True, port=8052)


C:\Users\NinaMayer\AppData\Local\Temp\ipykernel_22088\1551617559.py:127: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  focus_union = gdf_states_focus.unary_union


In [4]:
# -*- coding: utf-8 -*-
"""
Dash: Thüringen + Sachsen-Anhalt
- Hintergrund: grau (nur Fokusfläche)
- Brückenpunkte: farbig nach wählbarer Kennzahl (Zustand / Traglast / Alter / Substanz)
- Skala (Colorbar): GLOBAL über beide Bundesländer:
    cmin = bester (min) Wert aller Brücken in TH+SA
    cmax = schlechtester (max) Wert aller Brücken in TH+SA
- Optional: Bundesländer- und Landkreis-Grenzen einblendbar

Start:
    python bruecken_th_sa_points_globalminmax.py
Browser:
    http://127.0.0.1:8054
"""

import json
from pathlib import Path
from typing import List, Dict, Tuple, Optional

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import plotly.graph_objects as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output


# ------------------------------------------------------------
# Konfiguration
# ------------------------------------------------------------
csv_path = Path("data/original_bridge_statistic_germany.csv")
geo_path = Path("landkreise_simplify200.geojson")
states_path = Path("bundeslaender_simplify0.geojson")

AGE_REF_YEAR = 2025
FOCUS_STATES = ["Thüringen", "Sachsen-Anhalt"]  # exakt wie im GeoJSON


# ------------------------------------------------------------
# Helper
# ------------------------------------------------------------
def pick_first(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    return None


def make_points_from_lonlat(dfin, lon_col="x2", lat_col="y2"):
    lon = pd.to_numeric(dfin[lon_col].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    lat = pd.to_numeric(dfin[lat_col].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    ok = lon.notna() & lat.notna()
    g = gpd.GeoDataFrame(
        dfin[ok].copy(),
        geometry=[Point(xy) for xy in zip(lon[ok], lat[ok])],
        crs=4326,
    )
    g["lon"] = g.geometry.x
    g["lat"] = g.geometry.y
    return g


def make_points_from_utm(dfin, x_col="X", y_col="Y", epsg=25832):
    xx = pd.to_numeric(dfin[x_col].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    yy = pd.to_numeric(dfin[y_col].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    ok = xx.notna() & yy.notna()
    g = gpd.GeoDataFrame(
        dfin[ok].copy(),
        geometry=[Point(xy) for xy in zip(xx[ok], yy[ok])],
        crs=epsg,
    ).to_crs(4326)
    g["lon"] = g.geometry.x
    g["lat"] = g.geometry.y
    return g


def safe_minmax(series: pd.Series) -> Optional[Tuple[float, float]]:
    s = pd.to_numeric(series, errors="coerce").dropna()
    if len(s) == 0:
        return None
    vmin = float(s.min())
    vmax = float(s.max())
    if vmin == vmax:
        eps = 1e-6 if vmin == 0 else abs(vmin) * 1e-6
        vmin -= eps
        vmax += eps
    return (vmin, vmax)


# ------------------------------------------------------------
# 1) Geodaten laden + Fokus bilden
# ------------------------------------------------------------
gdf_kreise = gpd.read_file(geo_path)
gdf_states = gpd.read_file(states_path)

# CRS auf WGS84
if gdf_kreise.crs is None:
    gdf_kreise = gdf_kreise.set_crs(4326)
else:
    gdf_kreise = gdf_kreise.to_crs(4326)

if gdf_states.crs is None:
    gdf_states = gdf_states.set_crs(4326)
else:
    gdf_states = gdf_states.to_crs(4326)

# Spalten
colK_name = pick_first(gdf_kreise.columns, ["GEN", "NAME_3", "NAME", "GEN_NAME", "KREIS"])
colK_ags = pick_first(gdf_kreise.columns, ["AGS", "AGS_0", "RS", "RS_0", "ID_3"])
if colK_name is None:
    raise ValueError(f"Konnte keinen Kreisnamen finden. Spalten: {list(gdf_kreise.columns)}")

colS_name = pick_first(gdf_states.columns, ["GEN", "NAME", "STATE_NAME"])
colS_id = pick_first(gdf_states.columns, ["RS", "RS_0", "AGS", "AGS_0", "ID"])
if colS_name is None:
    raise ValueError(f"Konnte keinen Bundesländer-Namen finden. Spalten: {list(gdf_states.columns)}")
if colS_id is None:
    colS_id = colS_name

gdf_states = gdf_states.copy()
gdf_states["state_id"] = gdf_states[colS_id].astype(str)

# Fokus-Bundesländer
gdf_states_focus = gdf_states[gdf_states[colS_name].isin(FOCUS_STATES)].copy()
if len(gdf_states_focus) == 0:
    raise ValueError(
        f"FOCUS_STATES matcht nicht. Verfügbare Werte in '{colS_name}': "
        f"{sorted(gdf_states[colS_name].astype(str).unique())}"
    )

focus_union = gdf_states_focus.unary_union

# Fokus-Landkreise: robust (Innenpunkt liegt innerhalb Fokus)
rp = gdf_kreise.geometry.representative_point()
gdf_kreise_focus = gdf_kreise[rp.within(focus_union)].copy()

# Fokusfläche als grauer Hintergrund
focus_outline = gdf_states_focus.dissolve().reset_index(drop=True)
focus_outline["id"] = "focus"
geojson_focus = json.loads(focus_outline[["id", "geometry"]].to_json())

# Bundesländer-GeoJSON (nur Fokus)
geojson_states = json.loads(gdf_states_focus[["state_id", "geometry"]].to_json())

# Kreis-IDs + GeoJSON für Grenzen
if colK_ags and colK_ags in gdf_kreise_focus.columns:
    gdf_kreise_focus = gdf_kreise_focus.copy()
    gdf_kreise_focus["id"] = gdf_kreise_focus[colK_ags].astype(str)
else:
    gdf_kreise_focus = gdf_kreise_focus.copy()
    gdf_kreise_focus["id"] = gdf_kreise_focus[colK_name].astype(str)

geojson_kreise = json.loads(gdf_kreise_focus[["id", "geometry"]].to_json())


# ------------------------------------------------------------
# 2) Brückendaten laden + Punkte + Join auf Fokus-Kreise
# ------------------------------------------------------------
df = pd.read_csv(csv_path, sep=";", decimal=",", dtype=str, keep_default_na=False)

for sp in [
    "Zustandsnote", "Substanzkennzahl",
    "x2", "y2", "X", "Y",
    "Kreis", "Baujahr Überbau", "Traglastindex",
    "Baustoffklasse", "Bauwerksname"
]:
    if sp not in df.columns:
        df[sp] = ""

df["Zustandsnote"] = pd.to_numeric(df["Zustandsnote"].astype(str).str.replace(",", ".", regex=False), errors="coerce")
df["Substanzkennzahl"] = pd.to_numeric(df["Substanzkennzahl"].astype(str).str.replace(",", ".", regex=False), errors="coerce")
df["Baujahr Überbau"] = pd.to_numeric(df["Baujahr Überbau"].astype(str).str.replace(",", ".", regex=False), errors="coerce")

# Punkte bevorzugt x2/y2, sonst X/Y
gdf_pts = make_points_from_lonlat(df, "x2", "y2")
used_coords = "WGS84 (x2/y2)"
if len(gdf_pts) == 0:
    gdf_pts = make_points_from_utm(df, "X", "Y", epsg=25832)
    used_coords = "UTM32 (EPSG:25832)"

# Join Punkte -> Fokus-Kreise (filtert automatisch auf TH+SA-Innenfläche)
join_cols = [colK_name, "geometry"] if colK_ags is None else [colK_name, colK_ags, "geometry"]
joined = gpd.sjoin(
    gdf_pts,
    gdf_kreise_focus[join_cols],
    how="left",
    predicate="within",
)
joined = joined[joined[colK_name].notna()].copy()
gdf_pts_focus = gdf_pts.loc[joined.index].copy()

# Kennzahlen für Farbgebung (Punkte)
roman_map = {"I": 1, "II": 2, "III": 3, "IV": 4, "V": 5}
gdf_pts_focus["zust_num"] = pd.to_numeric(gdf_pts_focus["Zustandsnote"], errors="coerce")
gdf_pts_focus["subst_num"] = pd.to_numeric(gdf_pts_focus["Substanzkennzahl"], errors="coerce")
gdf_pts_focus["trag_num"] = gdf_pts_focus["Traglastindex"].astype(str).str.upper().str.strip().map(roman_map)
gdf_pts_focus["baujahr_num"] = pd.to_numeric(gdf_pts_focus["Baujahr Überbau"], errors="coerce")
gdf_pts_focus["alter_num"] = AGE_REF_YEAR - gdf_pts_focus["baujahr_num"]

# Hovertext
gdf_pts_focus["hover"] = (
    "<b>" + gdf_pts_focus["Bauwerksname"].astype(str) + "</b><br>"
    "Zustandsnote: " + gdf_pts_focus["zust_num"].round(2).astype(str) + "<br>"
    "Traglastindex: " + gdf_pts_focus["trag_num"].astype("Int64").astype(str) + "<br>"
    "Alter (Jahre): " + gdf_pts_focus["alter_num"].round(1).astype(str) + "<br>"
    "Substanzkennzahl: " + gdf_pts_focus["subst_num"].round(2).astype(str)
)

# GLOBAL Min/Max über beide Bundesländer (TH+SA) je Kennzahl
GLOBAL_SCALE: Dict[str, Optional[Tuple[float, float]]] = {
    "zust_num":  safe_minmax(gdf_pts_focus["zust_num"]),
    "trag_num":  safe_minmax(gdf_pts_focus["trag_num"]),
    "alter_num": safe_minmax(gdf_pts_focus["alter_num"]),
    "subst_num": safe_minmax(gdf_pts_focus["subst_num"]),
}


# ------------------------------------------------------------
# 3) Dash UI
# ------------------------------------------------------------
point_mode_options = [
    {"label": "Zustandsnote",        "value": "zust"},
    {"label": "Traglastindex",       "value": "trag"},
    {"label": "Durchschnittsalter",  "value": "alter"},
    {"label": "Substanzkennzahl",    "value": "subst"},
]

layer_options = [
    {"label": "Bundesländer-Grenzen", "value": "states"},
    {"label": "Landkreis-Grenzen",    "value": "kreise"},
]

app = dash.Dash(__name__)

app.layout = html.Div(
    style={"display": "flex", "height": "100vh", "fontFamily": "sans-serif"},
    children=[
        html.Div(
            style={"width": "22%", "padding": "10px", "borderRight": "1px solid #ccc", "overflowY": "auto"},
            children=[
                html.H3("Fokus"),
                html.Div(", ".join(FOCUS_STATES)),
                html.Hr(),
                html.H3("Brückenpunkte einfärben nach"),
                dcc.RadioItems(
                    id="point-mode",
                    options=point_mode_options,
                    value="zust",
                    labelStyle={"display": "block"},
                ),
                html.Hr(),
                html.H3("Grenzen"),
                dcc.Checklist(
                    id="layer-checklist",
                    options=layer_options,
                    value=["states", "kreise"],
                    labelStyle={"display": "block"},
                ),
                html.Hr(),
                html.Div(f"Koordinaten: {used_coords}", style={"fontSize": "12px", "color": "#555"}),
            ],
        ),
        html.Div(
            style={"width": "78%", "padding": "10px"},
            children=[dcc.Graph(id="map-figure", style={"height": "100%"})],
        ),
    ],
)


# ------------------------------------------------------------
# 4) Figure Builder
# ------------------------------------------------------------
def build_figure(point_mode: str, layers: List[str]):
    fig = go.Figure()

    # 0) Grauer Hintergrund (Fokusfläche)
    fig.add_trace(
        go.Choropleth(
            geojson=geojson_focus,
            locations=["focus"],
            z=[0],
            featureidkey="properties.id",
            showlegend=False,
            showscale=False,
            hoverinfo="skip",
            colorscale=[[0, "rgb(245,245,245)"], [1, "rgb(245,245,245)"]],
            marker_line_color="rgba(0,0,0,0)",
            marker_line_width=0,
        )
    )

    # 1) Brückenpunkte farbig mit GLOBALER Skala über TH+SA
    if point_mode == "trag":
        col = "trag_num"
        label = "Traglastindex"
        fmt = ".0f"
    elif point_mode == "alter":
        col = "alter_num"
        label = "Alter (Jahre)"
        fmt = ".1f"
    elif point_mode == "subst":
        col = "subst_num"
        label = "Substanzkennzahl"
        fmt = ".2f"
    else:
        col = "zust_num"
        label = "Zustandsnote"
        fmt = ".2f"

    pts = gdf_pts_focus.copy()
    pts = pts[pts[col].notna()].copy()

    rng = GLOBAL_SCALE.get(col)
    if len(pts) > 0 and rng is not None:
        vmin, vmax = rng

        fig.add_trace(
            go.Scattergeo(
                lon=pts["lon"],
                lat=pts["lat"],
                mode="markers",
                name=f"Brücken – {label}",
                text=pts["hover"],
                hovertemplate="%{text}<extra></extra>",
                marker=dict(
                    size=7.0,
                    opacity=1.0,
                    color=pts[col],
                    colorscale="RdYlGn_r",
                    cmin=vmin,                 # <-- bester Wert (min) über TH+SA
                    cmax=vmax,                 # <-- schlechtester Wert (max) über TH+SA
                    showscale=True,
                    colorbar=dict(
                        title=f"{label}<br>min={vmin:{fmt}} / max={vmax:{fmt}}",
                        thickness=12
                    ),
                    line=dict(width=0.3, color="rgba(0,0,0,0.35)"),
                ),
                showlegend=True,
            )
        )

    # 2) Bundesländer-Grenzen (weiß)
    if "states" in layers:
        fig.add_trace(
            go.Choropleth(
                geojson=geojson_states,
                locations=gdf_states_focus["state_id"],
                z=[0] * len(gdf_states_focus),
                featureidkey="properties.state_id",
                name="Bundesländer",
                showlegend=True,
                showscale=False,
                hoverinfo="skip",
                colorscale=[[0, "rgba(0,0,0,0)"], [1, "rgba(0,0,0,0)"]],
                marker_line_color="rgba(160,160,160,0.95)",
                marker_line_width=1.8,
            )
        )

    # 3) Landkreis-Grenzen (hellgrau)
    if "kreise" in layers:
        fig.add_trace(
            go.Choropleth(
                geojson=geojson_kreise,
                locations=gdf_kreise_focus["id"],
                z=[0] * len(gdf_kreise_focus),
                featureidkey="properties.id",
                showlegend=False,
                showscale=False,
                hoverinfo="skip",
                colorscale=[[0, "rgba(0,0,0,0)"], [1, "rgba(0,0,0,0)"]],
                marker_line_color="rgba(160,160,160,0.7)",
                marker_line_width=0.45,
            )
        )

    # Zoom
    fig.update_geos(
        fitbounds="locations",
        visible=False,
        projection_type="mercator",
    )

    fig.update_layout(
        margin={"r": 20, "t": 60, "l": 20, "b": 20},
        legend=dict(x=0.02, y=0.98, bgcolor="rgba(255,255,255,0.85)"),
        title=(
            f"Thüringen + Sachsen-Anhalt – Brückenpunkte nach {label}<br>"
            f"<sup>Skala: bester bis schlechtester Brückenwert über beide Länder. Koordinaten: {used_coords}</sup>"
        ),
        paper_bgcolor="rgb(245,245,245)",
        plot_bgcolor="rgb(245,245,245)",
    )

    return fig


@app.callback(
    Output("map-figure", "figure"),
    [Input("point-mode", "value"), Input("layer-checklist", "value")],
)
def update_map(point_mode, layers):
    if point_mode is None:
        point_mode = "zust"
    if layers is None:
        layers = []
    return build_figure(point_mode, layers)


if __name__ == "__main__":
    app.run(debug=True, port=8054)


C:\Users\NinaMayer\AppData\Local\Temp\ipykernel_22088\3498793174.py:134: DeprecationWarning:

The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.



In [3]:
# -*- coding: utf-8 -*-
"""
Dash: Thüringen + Sachsen-Anhalt
- Hintergrund: hellgraue Fokusfläche (nur TH+SA)
- Landkreise: Choropleth nach Ø Kennzahl der Brücken im Landkreis (aus Punktdaten aggregiert)
- Brückenpunkte: farbig nach wählbarer Kennzahl
- Skala (Colorbar): GLOBAL über beide Bundesländer (TH+SA) aus MIN/MAX der Brückenwerte
- Optional: Bundesländer- und Landkreis-Grenzen einblendbar

Start:
    python bruecken_th_sa_choro_plus_points_globalminmax.py
Browser:
    http://127.0.0.1:8054
"""

import json
import re
import unicodedata
from pathlib import Path
from typing import List, Dict, Tuple, Optional

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import plotly.graph_objects as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output


# ------------------------------------------------------------
# Konfiguration
# ------------------------------------------------------------
csv_path = Path("data/original_bridge_statistic_germany.csv")
geo_path = Path("landkreise_simplify200.geojson")
states_path = Path("bundeslaender_simplify0.geojson")

AGE_REF_YEAR = 2025
FOCUS_STATES = ["Thüringen", "Sachsen-Anhalt"]  # muss exakt wie im GeoJSON sein


# ------------------------------------------------------------
# Helper
# ------------------------------------------------------------
def pick_first(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    return None


def make_points_from_lonlat(dfin, lon_col="x2", lat_col="y2"):
    lon = pd.to_numeric(dfin[lon_col].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    lat = pd.to_numeric(dfin[lat_col].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    ok = lon.notna() & lat.notna()
    g = gpd.GeoDataFrame(
        dfin[ok].copy(),
        geometry=[Point(xy) for xy in zip(lon[ok], lat[ok])],
        crs=4326,
    )
    g["lon"] = g.geometry.x
    g["lat"] = g.geometry.y
    return g


def make_points_from_utm(dfin, x_col="X", y_col="Y", epsg=25832):
    xx = pd.to_numeric(dfin[x_col].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    yy = pd.to_numeric(dfin[y_col].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    ok = xx.notna() & yy.notna()
    g = gpd.GeoDataFrame(
        dfin[ok].copy(),
        geometry=[Point(xy) for xy in zip(xx[ok], yy[ok])],
        crs=epsg,
    ).to_crs(4326)
    g["lon"] = g.geometry.x
    g["lat"] = g.geometry.y
    return g


def safe_minmax(series: pd.Series) -> Optional[Tuple[float, float]]:
    s = pd.to_numeric(series, errors="coerce").dropna()
    if len(s) == 0:
        return None
    vmin = float(s.min())
    vmax = float(s.max())
    if vmin == vmax:
        eps = 1e-6 if vmin == 0 else abs(vmin) * 1e-6
        vmin -= eps
        vmax += eps
    return (vmin, vmax)


# ------------------------------------------------------------
# 1) Geodaten laden + Fokus bilden
# ------------------------------------------------------------
gdf_kreise = gpd.read_file(geo_path)
gdf_states = gpd.read_file(states_path)

# CRS auf WGS84
if gdf_kreise.crs is None:
    gdf_kreise = gdf_kreise.set_crs(4326)
else:
    gdf_kreise = gdf_kreise.to_crs(4326)

if gdf_states.crs is None:
    gdf_states = gdf_states.set_crs(4326)
else:
    gdf_states = gdf_states.to_crs(4326)

# Spalten bestimmen
colK_name = pick_first(gdf_kreise.columns, ["GEN", "NAME_3", "NAME", "GEN_NAME", "KREIS"])
colK_ags = pick_first(gdf_kreise.columns, ["AGS", "AGS_0", "RS", "RS_0", "ID_3"])
if colK_name is None:
    raise ValueError(f"Konnte keinen Kreisnamen finden. Spalten: {list(gdf_kreise.columns)}")

colS_name = pick_first(gdf_states.columns, ["GEN", "NAME", "STATE_NAME"])
colS_id = pick_first(gdf_states.columns, ["RS", "RS_0", "AGS", "AGS_0", "ID"])
if colS_name is None:
    raise ValueError(f"Konnte keinen Bundesländer-Namen finden. Spalten: {list(gdf_states.columns)}")
if colS_id is None:
    colS_id = colS_name

gdf_states = gdf_states.copy()
gdf_states["state_id"] = gdf_states[colS_id].astype(str)

# Fokus-Bundesländer
gdf_states_focus = gdf_states[gdf_states[colS_name].isin(FOCUS_STATES)].copy()
if len(gdf_states_focus) == 0:
    raise ValueError(
        f"FOCUS_STATES matcht nicht. Verfügbare Werte in '{colS_name}': "
        f"{sorted(gdf_states[colS_name].astype(str).unique())}"
    )

# Union der Fokus-Länder
focus_union = gdf_states_focus.unary_union

# Fokus-Landkreise: Innenpunkt muss in Fokusfläche liegen
rp = gdf_kreise.geometry.representative_point()
gdf_kreise_focus = gdf_kreise[rp.within(focus_union)].copy()

# Kreis-ID festlegen (stabil)
if colK_ags and colK_ags in gdf_kreise_focus.columns:
    gdf_kreise_focus = gdf_kreise_focus.copy()
    gdf_kreise_focus["id"] = gdf_kreise_focus[colK_ags].astype(str)
else:
    gdf_kreise_focus = gdf_kreise_focus.copy()
    gdf_kreise_focus["id"] = gdf_kreise_focus[colK_name].astype(str)

# GeoJSONs
geojson_kreise = json.loads(gdf_kreise_focus[["id", "geometry"]].to_json())
geojson_states = json.loads(gdf_states_focus[["state_id", "geometry"]].to_json())

# Fokusfläche als Hintergrund
focus_outline = gdf_states_focus.dissolve().reset_index(drop=True)
focus_outline["id"] = "focus"
geojson_focus = json.loads(focus_outline[["id", "geometry"]].to_json())


# ------------------------------------------------------------
# 2) Brückendaten laden + Punkte + Join auf Fokus-Kreise
# ------------------------------------------------------------
df = pd.read_csv(csv_path, sep=";", decimal=",", dtype=str, keep_default_na=False)

for sp in [
    "Zustandsnote", "Substanzkennzahl",
    "x2", "y2", "X", "Y",
    "Kreis", "Baujahr Überbau", "Traglastindex",
    "Baustoffklasse", "Bauwerksname"
]:
    if sp not in df.columns:
        df[sp] = ""

# numerische Felder
df["Zustandsnote"] = pd.to_numeric(df["Zustandsnote"].astype(str).str.replace(",", ".", regex=False), errors="coerce")
df["Substanzkennzahl"] = pd.to_numeric(df["Substanzkennzahl"].astype(str).str.replace(",", ".", regex=False), errors="coerce")
df["Baujahr Überbau"] = pd.to_numeric(df["Baujahr Überbau"].astype(str).str.replace(",", ".", regex=False), errors="coerce")

# Punkte bevorzugt WGS84 x2/y2, sonst UTM X/Y
gdf_pts = make_points_from_lonlat(df, "x2", "y2")
used_coords = "WGS84 (x2/y2)"
if len(gdf_pts) == 0:
    gdf_pts = make_points_from_utm(df, "X", "Y", epsg=25832)
    used_coords = "UTM32 (EPSG:25832)"

# Join: Punkte -> Fokus-Kreise (id muss mit)
joined = gpd.sjoin(
    gdf_pts,
    gdf_kreise_focus[["id", colK_name, "geometry"]],
    how="left",
    predicate="within",
)
joined = joined[joined["id"].notna()].copy()

gdf_pts_focus = gdf_pts.loc[joined.index].copy()
gdf_pts_focus["kreis_id"] = joined["id"].astype(str).values
gdf_pts_focus["kreis_name"] = joined[colK_name].astype(str).values

# Kennzahlen für Punkte
roman_map = {"I": 1, "II": 2, "III": 3, "IV": 4, "V": 5}
gdf_pts_focus["zust_num"] = pd.to_numeric(gdf_pts_focus["Zustandsnote"], errors="coerce")
gdf_pts_focus["subst_num"] = pd.to_numeric(gdf_pts_focus["Substanzkennzahl"], errors="coerce")
gdf_pts_focus["trag_num"] = gdf_pts_focus["Traglastindex"].astype(str).str.upper().str.strip().map(roman_map)
gdf_pts_focus["baujahr_num"] = pd.to_numeric(gdf_pts_focus["Baujahr Überbau"], errors="coerce")
gdf_pts_focus["alter_num"] = AGE_REF_YEAR - gdf_pts_focus["baujahr_num"]

# Hovertext
gdf_pts_focus["hover"] = (
    "<b>" + gdf_pts_focus["Bauwerksname"].astype(str) + "</b><br>"
    "Landkreis: " + gdf_pts_focus["kreis_name"].astype(str) + "<br>"
    "Zustandsnote: " + gdf_pts_focus["zust_num"].round(2).astype(str) + "<br>"
    "Traglastindex: " + gdf_pts_focus["trag_num"].astype("Int64").astype(str) + "<br>"
    "Alter (Jahre): " + gdf_pts_focus["alter_num"].round(1).astype(str) + "<br>"
    "Substanzkennzahl: " + gdf_pts_focus["subst_num"].round(2).astype(str)
)

# GLOBAL Min/Max über alle Brücken in TH+SA je Kennzahl
GLOBAL_SCALE: Dict[str, Optional[Tuple[float, float]]] = {
    "zust_num":  safe_minmax(gdf_pts_focus["zust_num"]),
    "trag_num":  safe_minmax(gdf_pts_focus["trag_num"]),
    "alter_num": safe_minmax(gdf_pts_focus["alter_num"]),
    "subst_num": safe_minmax(gdf_pts_focus["subst_num"]),
}

# Landkreis-Aggregation aus den Brückenpunkten (für Choropleth im Hintergrund)
KREIS_AGG = (
    gdf_pts_focus.dropna(subset=["kreis_id"])
    .groupby("kreis_id", as_index=False)
    .agg(
        n_bridges=("Bauwerksname", "size"),
        mean_zust=("zust_num", "mean"),
        mean_trag=("trag_num", "mean"),
        mean_alter=("alter_num", "mean"),
        mean_subst=("subst_num", "mean"),
    )
)

gdf_kreise_choro = gdf_kreise_focus.merge(KREIS_AGG, left_on="id", right_on="kreis_id", how="left")


# ------------------------------------------------------------
# 3) Dash UI
# ------------------------------------------------------------
point_mode_options = [
    {"label": "Zustandsnote",        "value": "zust"},
    {"label": "Traglastindex",       "value": "trag"},
    {"label": "Durchschnittsalter",  "value": "alter"},
    {"label": "Substanzkennzahl",    "value": "subst"},
]

layer_options = [
    {"label": "Landkreis-Flächen (Ø Wert)", "value": "choro"},
    {"label": "Brückenpunkte",              "value": "points"},
    {"label": "Bundesländer-Grenzen",       "value": "states"},
    {"label": "Landkreis-Grenzen",          "value": "kreise"},
]

app = dash.Dash(__name__)

app.layout = html.Div(
    style={"display": "flex", "height": "100vh", "fontFamily": "sans-serif"},
    children=[
        html.Div(
            style={"width": "24%", "padding": "10px", "borderRight": "1px solid #ccc", "overflowY": "auto"},
            children=[
                html.H3("Fokus"),
                html.Div(", ".join(FOCUS_STATES)),
                html.Hr(),
                html.H3("Einfärbung (Punkte + Flächen)"),
                dcc.RadioItems(
                    id="mode",
                    options=point_mode_options,
                    value="zust",
                    labelStyle={"display": "block"},
                ),
                html.Hr(),
                html.H3("Layer"),
                dcc.Checklist(
                    id="layers",
                    options=layer_options,
                    value=["choro", "points", "states", "kreise"],
                    labelStyle={"display": "block"},
                ),
                html.Hr(),
                html.Div(f"Koordinaten: {used_coords}", style={"fontSize": "12px", "color": "#555"}),
            ],
        ),
        html.Div(
            style={"width": "76%", "padding": "10px"},
            children=[dcc.Graph(id="map", style={"height": "100%"})],
        ),
    ],
)


# ------------------------------------------------------------
# 4) Figure Builder
# ------------------------------------------------------------
def build_figure(mode: str, layers: List[str]) -> go.Figure:
    fig = go.Figure()

    # Mode -> Spalten
    if mode == "trag":
        p_col, label, fmt, c_col = "trag_num", "Traglastindex", ".0f", "mean_trag"
    elif mode == "alter":
        p_col, label, fmt, c_col = "alter_num", "Alter (Jahre)", ".1f", "mean_alter"
    elif mode == "subst":
        p_col, label, fmt, c_col = "subst_num", "Substanzkennzahl", ".2f", "mean_subst"
    else:
        p_col, label, fmt, c_col = "zust_num", "Zustandsnote", ".2f", "mean_zust"

    rng = GLOBAL_SCALE.get(p_col)
    if rng is None:
        vmin, vmax = 0.0, 1.0
    else:
        vmin, vmax = rng

    # 0) Heller Hintergrund (Fokusfläche)
    fig.add_trace(
        go.Choropleth(
            geojson=geojson_focus,
            locations=["focus"],
            z=[0],
            featureidkey="properties.id",
            showlegend=False,
            showscale=False,
            hoverinfo="skip",
            colorscale=[[0, "rgb(248,248,248)"], [1, "rgb(248,248,248)"]],
            marker_line_color="rgba(0,0,0,0)",
            marker_line_width=0,
        )
    )

    # 1) Landkreis-Choropleth (Ø Wert) im Hintergrund (gleiche Skala wie Punkte)
    if "choro" in layers:
        gch = gdf_kreise_choro.copy()
        g_valid = gch[gch[c_col].notna()].copy()
        g_miss = gch[gch[c_col].isna()].copy()

        if len(g_miss) > 0:
            fig.add_trace(
                go.Choropleth(
                    geojson=geojson_kreise,
                    locations=g_miss["id"],
                    z=[0] * len(g_miss),
                    featureidkey="properties.id",
                    showlegend=False,
                    showscale=False,
                    hoverinfo="skip",
                    colorscale=[[0, "rgb(235,235,235)"], [1, "rgb(235,235,235)"]],
                    marker_opacity=1.0,
                    marker_line_color="rgba(0,0,0,0)",
                    marker_line_width=0.0,
                )
            )

        if len(g_valid) > 0:
            fig.add_trace(
                go.Choropleth(
                    geojson=geojson_kreise,
                    locations=g_valid["id"],
                    z=g_valid[c_col],
                    featureidkey="properties.id",
                    name=f"Landkreis Ø {label}",
                    showlegend=True,
                    showscale=False,  # Skala kommt von Punkten
                    colorscale="RdYlGn_r",
                    zmin=vmin,
                    zmax=vmax,
                    marker_opacity=0.78,
                    hovertemplate=(
                        "<b>%{text}</b><br>"
                        f"Ø {label}: %{{z:{fmt}}}<br>"
                        "Brücken (n): %{customdata}<extra></extra>"
                    ),
                    text=g_valid[colK_name],
                    customdata=g_valid["n_bridges"].fillna(0).astype(int),
                    marker_line_color="rgba(0,0,0,0)",
                    marker_line_width=0.0,
                )
            )

    # 2) Brückenpunkte farbig (mit Colorbar)
    if "points" in layers:
        pts = gdf_pts_focus.copy()
        pts = pts[pts[p_col].notna()].copy()

        if len(pts) > 0:
            fig.add_trace(
                go.Scattergeo(
                    lon=pts["lon"],
                    lat=pts["lat"],
                    mode="markers",
                    name=f"Brücken – {label}",
                    text=pts["hover"],
                    hovertemplate="%{text}<extra></extra>",
                    marker=dict(
                        size=6.5,
                        opacity=0.95,
                        color=pts[p_col],
                        colorscale="RdYlGn_r",
                        cmin=vmin,
                        cmax=vmax,
                        showscale=True,
                        colorbar=dict(
                            title=f"{label}<br>min={vmin:{fmt}} / max={vmax:{fmt}}",
                            thickness=14,
                            len=0.85,
                        ),
                        line=dict(width=0.35, color="rgba(0,0,0,0.35)"),
                    ),
                    showlegend=True,
                )
            )

    # 3) Bundesländer-Grenzen
    if "states" in layers:
        fig.add_trace(
            go.Choropleth(
                geojson=geojson_states,
                locations=gdf_states_focus["state_id"],
                z=[0] * len(gdf_states_focus),
                featureidkey="properties.state_id",
                name="Bundesländer",
                showlegend=True,
                showscale=False,
                hoverinfo="skip",
                colorscale=[[0, "rgba(0,0,0,0)"], [1, "rgba(0,0,0,0)"]],
                marker_line_color="rgba(160,160,160,0.95)",
                marker_line_width=2.0,
            )
        )

    # 4) Landkreis-Grenzen
    if "kreise" in layers:
        fig.add_trace(
            go.Choropleth(
                geojson=geojson_kreise,
                locations=gdf_kreise_focus["id"],
                z=[0] * len(gdf_kreise_focus),
                featureidkey="properties.id",
                showlegend=False,
                showscale=False,
                hoverinfo="skip",
                colorscale=[[0, "rgba(0,0,0,0)"], [1, "rgba(0,0,0,0)"]],
                marker_line_color="rgba(140,140,140,0.55)",
                marker_line_width=0.30,
            )
        )

    # Zoom/Geo
    fig.update_geos(
        fitbounds="locations",
        visible=False,
        projection_type="mercator",
        bgcolor="rgb(250,250,250)",
        showland=True,
        landcolor="rgb(248,248,248)",
    )

    fig.update_layout(
        margin={"r": 20, "t": 60, "l": 20, "b": 20},
        legend=dict(x=0.01, y=0.99, bgcolor="rgba(255,255,255,0.75)", font=dict(size=11)),
        title=(
            f"Thüringen + Sachsen-Anhalt – {label}: Landkreis Ø (Hintergrund) + Brückenpunkte<br>"
            f"<sup>Skala: min/max aller Brückenwerte in TH+SA. Koordinaten: {used_coords}</sup>"
        ),
        paper_bgcolor="rgb(250,250,250)",
        plot_bgcolor="rgb(250,250,250)",
    )
    return fig


@app.callback(
    Output("map", "figure"),
    [Input("mode", "value"), Input("layers", "value")],
)
def update_map(mode, layers):
    if mode is None:
        mode = "zust"
    if layers is None:
        layers = []
    return build_figure(mode, layers)


if __name__ == "__main__":
    app.run(debug=True, port=8054)


C:\Users\NinaMayer\AppData\Local\Temp\ipykernel_22088\2990224320.py:136: DeprecationWarning:

The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.

